In [1]:
import vk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print(u'Так надо')


In [1]:
token = 'token'

In [4]:
session = vk.Session(access_token = token) 
api = vk.API(session) 

In [5]:
group = 'cmcsovet'

In [6]:
#items = api.groups.getMembers(group_id = group, v = 5.73)['items']

In [7]:
from time import sleep

In [8]:
items = []

In [9]:
for i in range(0, int(pd.DataFrame(api.groups.getMembers(group_id = group, v = 5.73))['count'][0]/1000) + 1):
    res2 = api.groups.getMembers(group_id = group, count = 1000, offset = 1000 * i, v = 5.73)
    
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(10)

In [10]:
len(items)

1260

In [11]:
from tqdm import tqdm_notebook as tqdm

In [12]:
from tqdm import trange

In [13]:
dict_fr = {}
count_privat = 0
for i in tqdm(range(len(items))):
    try:
        dict_fr[i] = api.friends.get(user_id = items[i], v= 5.73, fields=['sex', 'city', 'university'])
    except: 
        count_privat+=1


In [14]:
len(dict_fr)

191

In [15]:
import json
import os
import networkx as nx

In [16]:
g = nx.Graph()

In [17]:
common_friends= dict()
coun = 0
for i in tqdm(range(len(items))):
    try:
        friends = dict_fr[i]['items']
        g.add_nodes_from((fr['id'], {'sex': fr.get('sex', -1), 
                    'first_name': fr.get('first_name', -1), 
                    'last_name': fr.get('last_name', -1)}) for fr in friends)

        for fr in friends:
            g.add_edge(items[i], fr['id'])
            try:
                city =fr['city']['title']
            except:
                city = -1
                
            nx.set_node_attributes(g, {fr['id']: city}, 'city')
            nx.set_node_attributes(g, {fr['id']: 0}, 'flag_dr')
    except KeyError:
        coun+=1

In [18]:
len(g.nodes)

52569

In [19]:
for item in tqdm(items):
    nx.set_node_attributes(g, {item: 1}, 'flag_dr')
    try:
        info = api.users.get(user_id = item, v = 5.73, fields = ['sex', 'city'])[0]
        nx.set_node_attributes(g, {item: info['sex']}, 'sex')
        nx.set_node_attributes(g, {item: info['city']['title']}, 'city')
    except:
        pass

In [20]:
len(g.edges)

66684

In [21]:
len(g.nodes)

52569

In [22]:
nx.write_graphml(g, 'cmcsovet.graphml')

In [23]:
nx.write_graphml(g, 'cmcsovet.gml')

В целом ввиду тематики сообщества "Студенческий совет ВМК МГУ" большинство участников сгруппировано в центре: многие знают друг друга

Алгоритм прорисовки

<center><img src='1.png', width=1200></center>

Добавьте изменение размера вершины в зависимости от степени вершины

<center><img src='2.png', width=1200></center>

Раскарсска по городам

<center><img src='3.png', width=350></center>

по городам очевидно все сгруппировано вокруг Москвы

<center><img src='4.png', width=1200></center>

раскраска по цвета

<center><img src='5.png', width=350></center>

по цветам

<center><img src='6.png', width=1200></center>

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [24]:
gg = nx.k_core(g, k =10)

In [25]:
len(gg.nodes), len(gg.edges)

(353, 4098)

In [26]:
degr_cent = nx.centrality.degree_centrality(gg)

In [27]:
df = pd.DataFrame()
df['id'], df['value'] = degr_cent.keys(), degr_cent.values()

In [28]:
id_degr = df[df['value']>0.12]['id'].values

In [29]:
for id_d in id_degr:
    nx.set_node_attributes(gg, {id_d: 1}, 'fl_degr')

In [31]:
closeness_nodes = nx.centrality.closeness_centrality(gg)

In [32]:
df_cent = pd.DataFrame()
df_cent['id'], df_cent['value'] = closeness_nodes.keys(), closeness_nodes.values()

In [33]:
len(df_cent[df_cent['value']>0.49])

41

In [34]:
id_cent = df_cent[df_cent['value']>0.49]['id'].values

In [35]:
for id_c in id_cent:
    nx.set_node_attributes(gg, {id_c: 1}, 'fl_cent')

In [37]:
betw_nodes = nx.betweenness_centrality(gg) 

In [44]:
df_betw = pd.DataFrame()
df_betw['id'], df_betw['value'] = betw_nodes.keys(), betw_nodes.values()

In [49]:
len(df_betw[df_betw['value']>0.008])

33

In [50]:
df_betw = df_betw[df_betw['value']>0.008]['id'].values

In [51]:
for id_c in df_betw:
    nx.set_node_attributes(gg, {id_c: 1}, 'fl_betw')

In [52]:
gg.nodes.data()

NodeDataView({23107592: {'sex': 2, 'first_name': 'Роман', 'last_name': 'Куцев', 'city': 'Москва', 'flag_dr': 1}, 51628040: {'sex': 2, 'first_name': 'Иван', 'last_name': 'Зотин', 'city': -1, 'flag_dr': 0}, 140335112: {'sex': 2, 'first_name': 'Егор', 'last_name': 'Задорин', 'city': 'Москва', 'flag_dr': 0}, 75501589: {'sex': 1, 'first_name': 'Линок', 'last_name': 'Марьина', 'city': 'Москва', 'flag_dr': 1}, 152551448: {'sex': 2, 'first_name': 'Андрей', 'last_name': 'Астапов', 'city': 'Москва', 'flag_dr': 1}, 75151388: {'sex': 1, 'first_name': 'Оля', 'last_name': 'Фортученко', 'city': -1, 'flag_dr': 0}, 19240995: {'sex': 1, 'first_name': 'Анна', 'last_name': 'Гончарова', 'city': 'Москва', 'flag_dr': 1}, 22788133: {'sex': 1, 'first_name': 'Александра', 'last_name': 'Подкина', 'city': 'Москва', 'flag_dr': 0}, 295653413: {'sex': 2, 'first_name': 'Career', 'last_name': 'Msu', 'city': 'Москва', 'flag_dr': 0}, 51873831: {'sex': 2, 'first_name': 'Александр', 'last_name': 'Линник', 'city': 'Королёв

In [53]:
nx.write_graphml(gg, 'cmcsovet_v2.graphml')

degree_centrality

<center><img src='7.png', width=1200></center>

<center><img src='8.png', width=1200></center>

closeness_centrality

<center><img src='9.png', width=1200></center>

<center><img src='10.png', width=1200></center>

betweenness_centrality

<center><img src='11.png', width=1200></center>

<center><img src='12.png', width=1200></center>